In [109]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
from hvplot import pandas
import string

################################################
def account(name):
    if name == "loan_balance" or "ire":
        return 111111
    else:
        print("account not found")
##################################################        

loan_data = Path("../fintech_collab/loan_data.xlsx")

loan_df = pd.read_excel(loan_data, header=0, index_col="Loan ID")

######################IMPORT CATEGORY DATASET #######################
ups_hfs_data = Path("../fintech_collab/category.xlsx")
ups_hfs_df =pd.read_excel(ups_hfs_data, header=0)



##############################################################################

#loan_df['Purchase_Date'] = (datetime.utcfromtimestamp(0) + timedelta(loan_df['Purchase Date']).strftime("%Y-%m-%d"))

loan_df.columns = loan_df.columns.str.strip()
loan_df.columns =loan_df.columns.str.replace(' ','_')

# fm_name = list(map(chr, range(97, 114)))
#can list be at cutting list of loan_df.columns[:3]
listA = ['Loan_Amount',
       'Beginning_Period_Principal_Outstanding',
       'Ending_Principal_Outstanding', 'Beginning_Period_Accrued_Interest',
       'Ending_Period_Accrued_Interest', 'Principal_Paid', 'Interest_Paid',
       'Late_Fee_Paid', 'Return_Item_Fee_Paid', 'Overpayment', 'Sundry_Loss',
       'Total_Payments_Received', 'Collection_Fees', 'Loan_Trailing_Fee',
       'Onboarding_Loans', 'Charge_offs']

# fm_name = list(map(chr, range(97, 114)))
# loop through the list to get total sum for each columns
j = 0

for i in listA:    
    i = loan_df[f"{i}"].sum()
    print(f" {listA[j]} is: {round(i,2):,}")
    listA[j] = round(i,2)
    j= j+1
    
        
#fm_name = chr(ord(fm_name) + 1)

#begining + onboarding loans - principal paid - charge off
cal_ending = listA[1]+ listA[14] - listA[5] - listA[-1]
print(f"Calculated ending is {cal_ending:,}")

######### CREATE TABLE for PRINCIPAL PAYMENT ########################

if listA[5] > 1:
    pmt_entry_df = pd.DataFrame({
        'AccountNumber': ['222222', account('loan_balance'),],
        'Amount':[listA[5], -listA[5]],
        'Description':['Principal Payment']},
        index = ['dr', 'cr'])
    
else:
    pmt_entry_df = pd.DataFrame({
    'AccountNumber': ['222222', account('loan_balance'),],
    'Amount':[-listA[5], listA[5]],
    'Description':['Principal Payment']},
    index = ['dr', 'cr'])
    
pmt_entry_df.head()

###########CREATE TABLE FOR INTEREST INCOME ENTRY ##################

int_entry_df2= pd.DataFrame({
    'AccountNumber': [account('irb'),'44444'],
    'Amount':[-3333, 3333],
    'Description':['Change in interest accrual']},
    index = ['dr', 'cr'])

int_entry_combined = pd.concat([pmt_entry_df, int_entry_df2], axis=0 )
int_entry_combined.head()
    
###########################################################################################

#loan_df = loan_df.groupby('Loan_Segment')['Ending Period Accrued Interest'].sum()
# loan_df.plot(x='Funded_Date', legend=True, stacked=True, rot=90)
#loan_df.plot.pie()
#loan_df

#ups_hfs_df.head()
# listB =loan_df.columns[:3]
# print(listB[0])

Left_join = pd.merge(loan_df, ups_hfs_df, on = 'Loan ID', how='left')
Left_join['Category_y'] = Left_join['Category_y'].fillna('HFI')

#Left_join= Left_join.groupby('Category_y')['Ending_Period_Accrued_Interest'].sum()
Left_join

 Loan_Amount is: 61,795,483
 Beginning_Period_Principal_Outstanding is: 47,927,569.16
 Ending_Principal_Outstanding is: 57,185,195.94
 Beginning_Period_Accrued_Interest is: 192,109.65
 Ending_Period_Accrued_Interest is: 307,101.75
 Principal_Paid is: 1,546,876.11
 Interest_Paid is: 447,726.19
 Late_Fee_Paid is: 8.63
 Return_Item_Fee_Paid is: 130
 Overpayment is: 1,156.56
 Sundry_Loss is: 0
 Total_Payments_Received is: 1,995,897.49
 Collection_Fees is: 1,467.12
 Loan_Trailing_Fee is: 3.61
 Onboarding_Loans is: 10,814,398.07
 Charge_offs is: 9,895.18
Calculated ending is 57,185,195.94


,Loan ID,Origination_Date,Purchase_Date,Sold_Date,Segment,Term,Status,Days_Past_Due,Loan_Amount,Beginning_Period_Principal_Outstanding,...,Category_x,Onboarding_Loans,Charge_offs,Loan_Segment,FICO_Range,Region,Gender,Employment_Type,Market_Channel,Category_y
0,131223,2020-01-15,2022-02-11,NaN,NEAR_PRIME,36,CLOSED,122,6000,0.00,...,HFI,NaN,0.0,Debt Consolidation,700-750,Mid West,Male,Other,Phone,HFS
1,971875,2021-12-30,2022-01-03,NaN,NEAR_PRIME,36,Current,0,25000,21611.50,...,HFS,NaN,0.0,Debt Consolidation,>750,Mid West,Male,Self-Employed,Online,HFS
2,112774,2021-12-30,2022-01-03,NaN,NEAR_PRIME,36,Current,0,10000,8949.94,...,HFS,NaN,0.0,Debt Consolidation,>750,Mid West,Male,Full-Time,Email-Campaign,HFS
3,659070,2022-01-10,2022-01-13,NaN,PRIME,60,Current,0,13000,12165.50,...,HFS,NaN,0.0,Debt Consolidation,<625,East,Male,Full-Time,Online,HFS
4,399956,2021-12-29,2022-01-03,NaN,NEAR_PRIME,36,Current,0,20593,18172.16,...,HFS,NaN,0.0,Debt Consolidation,650-700,North West,Male,Full-Time,Phone,HFS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,210908,2022-07-25,2022-07-28,NaN,NEAR_PRIME,36,Current,0,3000,0.00,...,HFI,3000.0,0.0,Other,625-650,North West,Male,Full-Time,Phone,HFI
4911,313775,2022-07-25,2022-07-28,NaN,NEAR_PRIME,48,Current,0,1500,0.00,...,HFI,1500.0,0.0,Other,<625,Mid West,Male,Part-Time,Phone,HFI
4912,242080,2022-07-25,2022-07-28,NaN,PRIME,36,Current,0,5000,0.00,...,HFI,5000.0,0.0,Other,<625,West,Male,Self-Employed,Phone,HFI
4913,305539,2022-07-25,2022-07-28,NaN,NEAR_PRIME,48,Current,0,7000,0.00,...,HFI,7000.0,0.0,Other,650-700,South,Male,Other,Online,HFI
